In [2]:
from typing import Dict, Any

import math            as math
import numpy           as np
import numpy.typing    as npt

#import plons scripts
import plons.PhysicalQuantities       as pq
import plons.ConversionFactors_cgs    as cgs
import plons.GeometricalFunctions     as gf

In [ ]:
from dotenv import load_dotenv
import os
from elasticsearch import helpers  # For bulk Data Uploading
from elasticsearch import Elasticsearch  # Base function for interacting with Elasticsearch

load_dotenv()
client = Elasticsearch("http://localhost:9200/", api_key=os.getenv('apikey'))

#test client
print(client.info())

In [ ]:
'''
"settings" is technically not needed if we are working on a simple local host, but can be changed to optimise search performance on a database that is hosted on a cluster and searched by multiple users.
"mappings" is required if you wish to explicitly map fields to specific values
'''

index_definition = {
    "settings": {  
        "number_of_shards": 1,
    },
    "mappings": {
        "properties": {
            "code hash": {"type": "keyword"},
            "path to folder": {"type": "keyword"},
            "simulation time": {
                "type": "float",
                "meta": {
                    "unit": "s",
                }
            },
            "particle mass": {
                "type": "float",
                "meta": {
                    "unit": "Msun",
                }
            },
            "tmax": {"type": "float"},
            "eccentricity": {"type": "float"},
            "semi-major axis": {
                "type": "float",
                "meta": {
                    "unit": "s",
                }
            },
            "wind mass rate": {
                "type": "float",
                "meta": {
                    "unit": "Msun/yr",
                }
            },
            "init wind velocity": {
                "type": "float",
                "meta": {
                    "unit": "km/s",
                }
            },
            "term wind velocity": {
                "type": "float",
                "meta": {
                    "unit": "km/s",
                }
            },
            "wind inject radius": {
                "type": "float",
                "meta": {
                    "unit": "au",
                }
            },
            "wind temperature": {
                "type": "float",
                "meta": {
                    "unit": "K",
                }
            },
        }
    }
}


### Load information from .setup and .in file




In [5]:
def LoadSetupData(dir: str, prefix: str) -> Dict[str, Any]:
    """ Load the prefix.in and prefix.setup files to get general information about the phantom model

    Args:
        dir (str): directory of the simulation
        prefix (str): prefix used for the files

    Returns:
        Dict[str, Any]: a dictionary containing the info from the setup files
        (!! check units, they are not all in SI or cgs)
    """

    # load the prefix.in & prefix.setup file
    setup = {}
    try:
        with open(os.path.join(dir,'%s.setup'%prefix), 'r') as f:
            lines = f.readlines()
            for string in lines:
                line = string.split()
                if len(line) != 0:
                    if line[0] != '#':
                        stringName = line[0]

                        # Names that have to be changed
                        # Floats
                        if stringName == 'primary_mass': stringName = 'primary mass'
                        elif stringName == 'wind_gamma' or stringName == "temp_exponent": stringName = 'gamma'
                        elif stringName == 'eccentricity': stringName = 'ecc'
                        # For triples
                        elif stringName == 'secondary_mass': stringName = 'massComp_ini'
                        elif stringName == 'semi_major_axis': stringName = 'sma_ini'
                        elif stringName == 'binary2_a' : stringName = 'sma_in_ini'
                        elif stringName == 'binary2_e' : stringName = 'ecc_in'
                        elif stringName == 'secondary_racc': stringName = 'rAccrComp'
                        elif stringName == 'accr2b' : stringName = 'rAccrComp_in'
                        elif stringName == 'racc2b' : stringName = 'rAccrComp_in'
                        # If name doesn't change
                        setup[stringName] = float(line[2])
                        # For triples
                        if stringName == 'q2':
                            stringName = 'massComp_in_ini'
                            setup[stringName] = float(line[2])*setup['primary mass']

                        # Boolean
                        if stringName == 'icompanion_star':
                            stringName = 'single_star'
                            if int(line[2]) == 0: setup[stringName] = True
                            else: setup[stringName] = False
                            stringName = 'triple_star'
                            if int(line[2]) == 2: setup[stringName] = True
                            else: setup[stringName] = False

    except FileNotFoundError:
        print('')
        print(" ERROR: No %s.setup file found!"%prefix)
        print('')
        exit()

    try:
        with open(os.path.join(dir,'%s.in'%prefix), 'r') as f:
            lines = f.readlines()
            for string in lines:
                line = string.split()
                if len(line) != 0:
                    if line[0] != '#':
                        stringName = line[0]

                        # Strings
                        if stringName == 'logfile': setup[stringName] = str(line[2])
                        elif stringName == 'dumpfile': setup[stringName] = str(line[2])
                        elif stringName == 'twallmax': setup[stringName] = str(line[2])
                        elif stringName == 'dtwallmax': setup[stringName] = str(line[2])

                        # Floats
                        else:
                            if stringName == 'wind_velocity': stringName = 'init wind velocity'
                            elif stringName == 'outer_boundary': stringName = 'outer boundary'
                            elif stringName == 'wind_mass_rate': stringName = 'wind mass rate'
                            if line[2]=='F': line[2] = 0
                            setup[stringName] = float(line[2])

    except FileNotFoundError:
        print('')
        print(" ERROR: No %s.in file found!"%prefix)
        print('')
        exit()
    
    if setup["single_star"] == True:
        setup["sma_ini"] = 0.

    return setup


In [4]:
Dir = "/lhome/jolienm/Documents/TierModels/finalModelsHIcooling/v10e00"
prefix ="wind"
LoadSetupData(Dir,prefix)

{'primary mass': 1.5,
 'primary_racc': 1.22,
 'primary_lum': 5315.0,
 'primary_Teff': 3000.0,
 'primary_Reff': 1.26675855,
 'icompanion_star': 1.0,
 'single_star': False,
 'triple_star': False,
 'massComp_ini': 1.0,
 'rAccrComp': 0.04,
 'secondary_lum': 0.0,
 'secondary_Teff': 0.0,
 'secondary_Reff': 0.0,
 'sma_ini': 6.0,
 'ecc': 0.0,
 'mass_of_particles': 1e-11,
 'gamma': 1.2,
 'logfile': 'wind02.log',
 'dumpfile': 'wind_00600',
 'tmax': 600.0,
 'dtmax': 1.0,
 'nmax': -1.0,
 'nout': -1.0,
 'nmaxdumps': -1.0,
 'twallmax': '000:00',
 'dtwallmax': '012:00',
 'nfulldump': 1.0,
 'iverbose': 0.0,
 'C_cour': 0.3,
 'C_force': 0.25,
 'tolv': 0.01,
 'hfact': 1.2,
 'tolh': 0.0001,
 'alpha': 1.0,
 'alphamax': 1.0,
 'alphau': 1.0,
 'beta': 2.0,
 'avdecayconst': 0.1,
 'idamp': 0.0,
 'ieos': 2.0,
 'mu': 2.38,
 'ipdv_heating': 1.0,
 'ishock_heating': 1.0,
 'C_cool': 0.05,
 'icooling': 1.0,
 'icool_method': 0.0,
 'excitation_HI': 1.0,
 'relax_bowen': 0.0,
 'relax_stefan': 0.0,
 'dust_collision': 0.0,


In [ ]:

# def gen_data(image_number: int):
#     return {
#         "git hash": "8743b52063cd84097a65d1633f5c74f5",
#         "resolution": random.choice([64, 128, 256, 512]),
#         "time": random.uniform(2000.0, 50000.0),
#         "path to image": f"/path/to/image_{image_number}.png",
#         "eccentricity": random.uniform(0.0, 1.0),
#         "semi-major axis": random.uniform(1.0, 10.0),
#         "primary mass": random.uniform(1000.0, 100000.0),
#         "primary racc": random.uniform(1.0, 10.0),
#         "primary Teff": random.uniform(100.0, 1000.0),
#         "primary Reff": random.uniform(1.0, 5.0),
#         "secondary mass": random.uniform(10.0, 1000.0),
#         "secondary racc": random.uniform(0.0, 1.0),
#         "secondary Teff": random.uniform(1.0, 100.0),
#         "secondary Reff": random.uniform(0.0, 1.0),
#         "adiabatic index": random.uniform(1.0, 10.0),
#         "mean molecular weight": random.uniform(1.0, 50.0),
#         "icooling": int(random.uniform(20.0, 50.0)),
#     }